<span style="color:#ffd33d; font-size:150%"> 2023.08.27: BIVA_08  </span>
## 2023.08.29

## 2023.09.05

BIVA-ver8.5

in seq 15
varialbe 102

In [1]:
import torch

print(torch.__version__)

2.2.1+cu121


In [2]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
!git clone https://github.com/hugwolf77/SVU_SCT_FIN.git

Cloning into 'SVU_SCT_FIN'...
remote: Enumerating objects: 1368, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 1368 (delta 7), reused 16 (delta 7), pack-reused 1352
Receiving objects: 100% (1368/1368), 22.62 MiB | 20.60 MiB/s, done.
Resolving deltas: 100% (921/921), done.


In [ ]:
%cd /content/SVU_SCT_FIN/Model/BIVA/

/content/SVU_SCT_FIN/Model/BIVA


In [ ]:
# import argparse
import os
import sys
# sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))
import io
from pickle import FALSE

import time
from datetime import date, datetime
from dataclasses import dataclass, field

from typing import Set, List, Dict, Tuple #, final

import torch
import random
import numpy as np
import pandas as pd

from model.exp_main import Exp_Main
from model.utils.metrics import *
from sklearn.preprocessing import StandardScaler
# from scipy.stats import pearsonr
# from sklearn.metrics import *

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.font_manager as fm

import seaborn as sns
import IPython
import IPython.display
mpl.rcParams['figure.figsize'] = (16,12)
mpl.rcParams['axes.grid'] = False
# %matplotlib inline


In [ ]:

fix_seed = 2023
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

# argparser 에서 class 형태로 변경.
class args:
    pass

save_path = '/content/drive/MyDrive/ZZ/Code_02/exp/v8/'

#-----------------------------------------------------------------------------------------------------------------------
result_df = pd.DataFrame(columns=['no','model_id','root','len_seq','len_pred',
                                  'learn_rate','batch','MSE','MAE','RMSE','MAPE'])

no = 0

len_seq_list = [15]
len_pred_list = [3]
batch_list = [3]
lr_rate_list = [0.0005]

#-----------------------------------------------------------------------------------------------------------------------

for len_seq in len_seq_list:
  for len_pred in len_pred_list:
    for batch in batch_list:
      for lr_rate in lr_rate_list:
        no += 1

        now = time.localtime()
        time_now = time.strftime('%Y%m%d', now)

        args.is_training = 1  # 'status'
        args.model = 'BIVA'   # 'model name'
        args.model_id = '{}-no{}-dt{}-seq{}-pred{}-bt{}-lr{}-mv7-kxre000'.format(args.model,no,time_now,len_seq,len_pred,batch,lr_rate)

        # dataset
        args.data = 'BIVA'                                                                                             # 'dataset type : data_loader'
        args.root_path = '/content/SVU_SCT_FIN/Model/BIVA/Base_data/'
        args.data_path = 'dataset_03_S.xlsx'

        args.des = 'Exp'                                                                                                 # 'exp description'
        args.loss = 'mse'                                                                                                # loss function
        args.lradj = '1'                                                                                                 # adjusting learning rate option
        args.use_amp = False                                                                                             # 'use automatic mixed precision training'

        args.features = 'MS'                                                                                             # 'forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate'
        args.target = 'A1' # 'GDP'                                                                                       # 'target feature in S or MS task')
        args.freq = 'M'                                                                                                  # 'freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h'
        args.checkpoints = '/content/drive/MyDrive/ZZ/Code_02/exp/v8/(BIVA)_checkpoints/'                                   #'location of model checkpoints'

        # forecasting task
        args.seq_len = len_seq                                                                                           # 'input sequence length'
        args.pred_len = len_pred
        args.label_len = 1

        args.channels = 96 + 6
        args.rnn_hid_size = 256
        args.imputed_weight = 1
        args.vae_hid_size = 256   # deprivation
        args.vae_latent_size = 16
        args.infer_hid_size = 128
        args.moving_avg = 7                                                                                    # 'window size of moving average'
        # args.moving_avg = [6,10,15,20,25,30,35,40,45,60]                                                              # 'window size' list of moving average'

        args.cols = None

        #
        args.conv1d = True
        args.conv_kernal = 1
        args.RIN = False
        args.combination = True

        args.last_chkpt = 'BIVA-no1-dt20230905-seq15-pred3-bt1-lr0.001-mv7-kxre01_last_checkpoint.pth'

        args.learning_rate = lr_rate
        args.batch_size = batch
        args.train_epochs = 150           # 'train epochs'
        args.patience = 150               # 'early stopping patience'

        args.embed = 'timeF'            # 'time features encoding, options:[timeF, fixed, learned]'
        args.inverse = False
        # args.activation = 'relu'        # 'activation'
        args.do_predict = False         # 'whether to predict unseen future data')

        # optimization
        args.num_workers = 10            # 'data loader num workers'
        args.itr = 1                    # 'experiments times'

        # GPU
        args.use_gpu = True             # 'use gpu'
        args.gpu = 0                    # 'gpu'
        args.use_multi_gpu = False      # 'use multiple gpus'
        args.devices = '0,1,2,3'        # 'device ids of multile gpus'
        # args.test_flop = False          # 'See utils/tools for usage'

        args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False
        if args.use_gpu and args.use_multi_gpu:
            args.dvices = args.devices.replace(' ', '')
            device_ids = args.devices.split(',')
            args.device_ids = [int(id_) for id_ in device_ids]
            args.gpu = args.device_ids[0]

        print('Args in experiment:')
        print(args)
        #-----------------------------------------------------------------------------------------------------------------------
        print(f"\n train start -----------------------> {no}\n")
        #-----------------------------------------------------------------------------------------------------------------------

        if args.is_training:
            start_time = time.time()
            for ii in range(args.itr):
                # setting record of experiments
                setting = '{}_{}_seq{}_pred{}_bt{}_lr{}_{}'.format(
                    args.model_id,
                    args.data,
                    args.seq_len,
                    args.pred_len,
                    args.batch_size,
                    args.learning_rate,
                    args.channels,
                    args.des, ii)

                exp = Exp_Main(args)  # set experiments
                print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                exp.train(setting)

                print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                preds, trues, inputx, mae, mse, rmse, mape, mspe, rse, corr = exp.test(setting)

                if args.do_predict:
                    print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
                    exp.predict(setting, True)

                time_cost = str(time.time() - start_time)

                result = {'no':no+1,'Model_id':args.model_id,'root':args.root_path,'len_seq':args.seq_len,'len_pred':args.pred_len,
                          'learn_rate':args.learning_rate,'batch':args.batch_size,
                          'MSE':mse,'MAE':mae,'RMSE':rmse, 'MAPE':mape,'Corr':corr, 'time_cost': time_cost}
                torch.cuda.empty_cache()
        else:
            ii = 0
            setting = '{}_{}_seq{}_pred{}_bt{}_lr{}_{}'.format(
                    args.model_id,
                    args.data,
                    args.seq_len,
                    args.pred_len,
                    args.batch_size,
                    args.learning_rate,
                    args.channels,
                    args.des, ii)

            exp = Exp_Main(args)  # set experiments
            print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            preds, trues, inputx, mae, mse, rmse, mape, mspe, rse, corr = exp.test(setting, test=1)

            result = {'no':no+1,'Model_id':args.model_id,'root':args.root_path,'len_seq':args.seq_len,'len_pred':args.pred_len,
                          'learn_rate':args.learning_rate,'batch':args.batch_size,
                          'MSE':mse,'MAE':mae,'RMSE':rmse, 'MAPE':mape,'Corr':corr} #, 'time_cost': time_cost}
            torch.cuda.empty_cache()


        result_df = result_df.append(result, ignore_index = True)
        result_df.to_csv(os.path.join(save_path,'(BIVA_v8.5C)_{}_CV_no{}_expKX02_pred_3.csv'.format(args.model_id,no)))
        print(f"\n train done -----------------------> {no} ---- {time_cost} \n")
        no += 1



Args in experiment:
<class '__main__.args'>

 train start -----------------------> 1

Use GPU: cuda:0
>>>>>>>start training : BIVA-no1-dt20231215-seq15-pred3-bt3-lr0.0005-mv7-kxre000_BIVA_seq15_pred3_bt3_lr0.0005_102>>>>>>>>>>>>>>>>>>>>>>>>>>
train 211
val 30
test 23
=========> load_last chkpt: BIVA-no1-dt20230905-seq15-pred3-bt1-lr0.001-mv7-kxre01_last_checkpoint.pth
	iters: 10, epoch: 1 | loss: 1.3083380 
	speed: 1.8516s/iter; left time: 19424.9175s
	iters: 20, epoch: 1 | loss: 1.2384925 
	speed: 1.1434s/iter; left time: 11984.1838s
	iters: 30, epoch: 1 | loss: 1.2407244 
	speed: 1.0990s/iter; left time: 11507.2735s


KeyboardInterrupt: ignored